In [1]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for catboost from https://files.pythonhosted.org/packages/6d/cc/bc14c8580740f05159fb0d2245978672eff9b2994c502f6b8523727b62f4/catboost-1.2.2-cp311-cp311-manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 307.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.7 MB/s eta 0:00:0000:0100:01


In [2]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
train = pd.read_csv('house_train.csv',index_col=0)
test = pd.read_csv('house_test.csv',index_col=0)

In [22]:
train = train.fillna(0)
test = test.fillna(0)

In [5]:
cat_features = list(set(train.columns) - set(train.describe().columns))

In [6]:
X = train.drop('SalePrice',axis=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, train['SalePrice'], test_size=0.2, random_state=42)

In [8]:
model = CatBoostRegressor(
    iterations=1e4, depth=5,
    loss_function='RMSE', verbose=True,metric_period=int(1e3),
    cat_features=cat_features)

model.fit(X_train, y_train,eval_set=(X_test,y_test))

Learning rate set to 0.012242
0:	learn: 75978.3967655	test: 82709.7039676	best: 82709.7039676 (0)	total: 55.4ms	remaining: 9m 13s
1000:	learn: 16171.2499397	test: 30187.3927211	best: 30187.3927211 (1000)	total: 3.62s	remaining: 32.6s
2000:	learn: 12467.0952063	test: 29412.0111767	best: 29412.0111767 (2000)	total: 7.46s	remaining: 29.8s
3000:	learn: 9929.6194861	test: 28915.7781203	best: 28915.7781203 (3000)	total: 11.6s	remaining: 27.2s
4000:	learn: 8294.5630444	test: 28676.3414668	best: 28676.3414668 (4000)	total: 15.4s	remaining: 23.2s
5000:	learn: 7082.9674714	test: 28484.9213910	best: 28484.9213910 (5000)	total: 19.3s	remaining: 19.3s
6000:	learn: 6190.4999649	test: 28316.9864780	best: 28316.9864780 (6000)	total: 23.1s	remaining: 15.4s
7000:	learn: 5454.1474190	test: 28214.5220443	best: 28214.5220443 (7000)	total: 26.8s	remaining: 11.5s
8000:	learn: 4855.1143125	test: 28124.3643590	best: 28124.3643590 (8000)	total: 30.6s	remaining: 7.64s
9000:	learn: 4340.2570656	test: 28089.845584

In [9]:
importances = model.get_feature_importance()
feature_importances = pd.Series(importances, index=X_train.columns).sort_values(ascending=False).reset_index()

In [10]:
trimmed_feat = list(feature_importances[feature_importances[0] >= 2]['index'])
cat_features = list(set(train[trimmed_feat].columns) - set(train[trimmed_feat].describe().columns))

In [11]:
model = CatBoostRegressor(
    iterations=1e4, depth=5,
    loss_function='RMSE', verbose=True,metric_period=int(1e3),
    cat_features=cat_features)

model.fit(X_train[trimmed_feat], y_train, eval_set=(X_test[trimmed_feat],y_test))

Learning rate set to 0.012242
0:	learn: 75959.0141205	test: 82685.3766728	best: 82685.3766728 (0)	total: 1.62ms	remaining: 16.2s


1000:	learn: 18233.4337779	test: 29055.8318342	best: 29055.8318342 (1000)	total: 771ms	remaining: 6.93s
2000:	learn: 14668.2819501	test: 28807.9680623	best: 28807.9680623 (2000)	total: 1.5s	remaining: 6.01s
3000:	learn: 12592.7562536	test: 28971.9900533	best: 28807.9680623 (2000)	total: 2.25s	remaining: 5.24s
4000:	learn: 10945.4164106	test: 29019.8720475	best: 28807.9680623 (2000)	total: 3.01s	remaining: 4.52s
5000:	learn: 9581.0869812	test: 29074.0580552	best: 28807.9680623 (2000)	total: 3.77s	remaining: 3.77s
6000:	learn: 8461.4195742	test: 29202.9624973	best: 28807.9680623 (2000)	total: 4.54s	remaining: 3.02s
7000:	learn: 7486.1240061	test: 29290.0559693	best: 28807.9680623 (2000)	total: 5.32s	remaining: 2.28s
8000:	learn: 6717.1787566	test: 29362.9042918	best: 28807.9680623 (2000)	total: 6.1s	remaining: 1.52s
9000:	learn: 6055.0655879	test: 29414.4958777	best: 28807.9680623 (2000)	total: 6.88s	remaining: 764ms
9999:	learn: 5515.2837423	test: 29445.5946416	best: 28807.9680623 (2000

In [12]:
predictions = model.predict(X_test[trimmed_feat])

In [13]:
len(predictions)

219

In [14]:
mean_squared_error(y_test, predictions, squared=False)

28807.968062252257

# Валидация (Эрдэни)

In [23]:
y_pred = model.predict(test[trimmed_feat])

In [16]:
df_true = pd.read_csv("house_y_true.csv")
y_true = df_true['SalePrice'].values
len(y_true)

365

In [24]:
mean_squared_error(y_true, y_pred, squared=False)

28107.736635064302